In [10]:
import re
import string
import pickle
import joblib

import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC

In [2]:
def remove_xml_tags(text):
    pattern = r"<.*?>"
    return re.sub(pattern, " ", text)

def remove_punct(text):
    table = str.maketrans({key: " " for key in string.punctuation})
    return text.translate(table)

def clean_text(text):
    # removes xml tags and punctuation
    # (in that order, bc <> are used to find xml tags)
    # and keeps only one space in a row
    return re.sub(
        "\s+",
        " ",
        remove_punct(
            remove_xml_tags(
                text.lower()
            )
        )
    ).strip()

In [5]:
from tqdm.notebook import tqdm

FIELDS = [
    "Art",
    "Biology",
    "Business",
    "Chemistry",
    "Geology",
    "Humanities",
    "Math",
    "Medicine",
    "Physics",
    "Psychology",
    "Social",
    "Tech"
]
field_svcs = {
    field: pickle.load(open(f"field_svc_en_full_data/field_{field}.pkl", "rb"))
    for field in tqdm(FIELDS)
}

In [6]:
vectorizer = joblib.load("vectorizer_full.pkl")

In [14]:
def predict_field(text):
    transformed = vectorizer.transform([clean_text(text)])
    predictions = {}
    for field, svc in field_svcs.items():
        predictions[field] = svc.predict_proba(transformed)[0][1]
    return predictions

In [17]:
text = "The dominant sequence transduction models are based on complex recurrent or convolutional neural networks in an encoder-decoder configuration. The best performing models also connect the encoder and decoder through an attention mechanism. We propose a new simple network architecture, the Transformer, based solely on attention mechanisms, dispensing with recurrence and convolutions entirely. Experiments on two machine translation tasks show these models to be superior in quality while being more parallelizable and requiring significantly less time to train. Our model achieves 28.4 BLEU on the WMT 2014 English-to-German translation task, improving over the existing best results, including ensembles by over 2 BLEU. On the WMT 2014 English-to-French translation task, our model establishes a new single-model state-of-the-art BLEU score of 41.8 after training for 3.5 days on eight GPUs, a small fraction of the training costs of the best models from the literature. We show that the Transformer generalizes well to other tasks by applying it successfully to English constituency parsing both with large and limited training data."

In [19]:
#%%time
predict_field(text)

{'Art': 0.02619606320670933,
 'Biology': 0.0202904159786579,
 'Business': 0.004704699267283964,
 'Chemistry': 0.08391154872454226,
 'Geology': 0.0010244492484677073,
 'Humanities': 0.012127997351935631,
 'Math': 0.08715545037870429,
 'Medicine': 0.12722607902983302,
 'Physics': 0.07080327081284618,
 'Psychology': 0.0028381707704305844,
 'Social': 0.011290741314430816,
 'Tech': 0.654763359150955}